# Write an App with PromptFlow SDK that leverages a complex flow with multiple steps and Tools 

In this part of the workshop we will build a more sophisticated app leveraging different flow types and tools on promptflow...

## Create a Flow 
You can create LLM apps using a Python function or class as the entry point, which encapsulating your app logic. You can directly test or run these entries with pure code experience.

In [4]:
# Confirm Promptflow is installed

!pf -v


{
  "promptflow": "1.11.0",
  "promptflow-core": "1.11.0",
  "promptflow-devkit": "1.11.0",
  "promptflow-tracing": "1.11.0"
}

Executable '/opt/homebrew/opt/python@3.11/bin/python3.11'
Python (Darwin) 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]


In [7]:
# Create a new flow...

!pf flow init --flow flow01

Creating flow from scratch...
Creating hello.py...
Creating data.jsonl...
Creating .promptflow folder...
Creating __pycache__ folder...
Creating flow.dag.yaml...
Creating hello.jinja2...
Creating /Users/ozgurguler/Developer/Projects/aistudio-workshop-ozguler/code/flow01/requirements.txt...
Creating /Users/ozgurguler/Developer/Projects/aistudio-workshop-ozguler/code/flow01/.gitignore...
Done. Created standard flow folder: /Users/ozgurguler/Developer/Projects/aistudio-workshop-ozguler/code/flow01.
You can execute this command to test the flow, pf flow test --flow flow01 --input flow01/data.jsonl


When we run "flow init", it creates a folder with name matching flows name (in this case flow01 folder) with necessary files in it.

![Alt text](../media/80.png)

Structure of the flow folder:

- flow.dag.yaml: The flow definition with inputs/outputs, nodes, tools and variants for authoring purpose. \
It defines the structure and logic of the flow, specifying how different tools are connected and how data is passed between them. \
While defining the flow implicitly in app.py is possible, using a flow.yaml file provides several benefits, including better separation of concerns,\
improved readability and maintainability, enhanced reusability and modularity, and a more declarative approach to defining workflows.\
This separation allows for a cleaner, more organized,


- .promptflow/flow.tools.json:  contains the tools meta referenced in flow.dag.yaml.
- Source code files (.py, .jinja2): User managed, the code scripts referenced by **tools**.
- requirements.txt: Python package dependencies for this flow.

*(If the PromptFlow extension is installed, you can do CMD + K, V to visualise the flow as in the PromptFlow web UI like below.)*

![Alt text](../media/81.png)

We now have the flow skeleton in place, previously created with the *'flow init'* command. \
We will modify the flow so that it has a Python tool step which runs the Prompty file reflecting the recent updates to prompt management in Build,24. \
Let's now update the flow.dag.yaml so that it simply leverages a **python tool** that loads and runs the prompty.prompty. 

Below is the prompty file for creative writing. (make sure your AOAI endpoint crendtials and base url are added to the .env file in the folder where prompty file is stored.)
creative_writing_assistant.prompty is saved under code/flow01 which you can copy from.


![Alt text](../media/82.png)